In [1]:
try:
    sc.stop()
except:
    pass
#---------------------------#
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder \
    .master('spark://master:7077') \
    .appName('PySpark-Kagggle-AmazonBook') \
    .getOrCreate()

In [2]:
csvfilePath="hdfs://master:9000/Data_Sample/Sample_1000_amazon-sales-rank-data-for-print-and-kindle-books/amazon_com_extras.csv"

csvDF = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(csvfilePath).cache()

csvDF.printSchema()

root
 |-- ASIN: string (nullable = true)
 |-- GROUP: string (nullable = true)
 |-- FORMAT: string (nullable = true)
 |-- TITLE: string (nullable = true)
 |-- AUTHOR: string (nullable = true)
 |-- PUBLISHER: string (nullable = true)



In [3]:
HadoopOutPath1 = "hdfs://master:9000/xgm/output/KaggleAmazon_BeginEnd04/part-00000"

OutDF1 = spark.read.options(header='false', inferschema='true',delimiter='\t').csv(HadoopOutPath1).toDF("ASIN","start_time","end_time").cache()

In [4]:
join_DF1 = csvDF.join(OutDF1,csvDF.ASIN == OutDF1.ASIN).drop(OutDF1.ASIN)

In [5]:
join_DF1.show()

+----------+-----+---------+--------------------+--------------------+--------------------+-------------------+-------------------+
|      ASIN|GROUP|   FORMAT|               TITLE|              AUTHOR|           PUBLISHER|         start_time|           end_time|
+----------+-----+---------+--------------------+--------------------+--------------------+-------------------+-------------------+
|022640014X| book|hardcover|The Diversity Bar...|  Natasha K. Warikoo|University Of Chi...|2017-01-01_18:00:00|2018-06-30_10:00:00|
|022640708X| book|hardcover|Seven Ways of Loo...|     Scott Samuelson|University Of Chi...|2017-11-11_05:00:00|2018-06-30_10:00:00|
|022655645X| book|hardcover|The Human Body in...|Stefanos Geroulan...|University Of Chi...|2017-12-12_02:00:00|2018-06-30_00:00:00|
|022643303X| book|hardcover|Canine Confidenti...|         Marc Bekoff|University Of Chi...|2017-11-11_05:00:00|2018-06-30_10:00:00|
|022618871X| book|hardcover|Plankton: Wonders...|    Christian Sardet|Univer

In [6]:
join_DF1.groupBy("GROUP").count().show()

+------+-----+
| GROUP|count|
+------+-----+
|kindle|    3|
|  book|  961|
+------+-----+



In [7]:
join_DF1.groupBy("FORMAT").count().show()

+--------------------+-----+
|              FORMAT|count|
+--------------------+-----+
|      kindle edition|    3|
|           paperback|  581|
|           hardcover|  368|
|mass market paper...|   12|
+--------------------+-----+



In [8]:
join_DF1.groupBy("PUBLISHER").count().show()

+--------------------+-----+
|           PUBLISHER|count|
+--------------------+-----+
|AVON, a division ...|    5|
|     Crown Archetype|    1|
|Interracial Multi...|    1|
|       Harpercollins|    1|
|Harcourt Brace Co...|    1|
|HarperCollins Pub...|    5|
|             Picador|    2|
|    Penguin Classics|    2|
|       Balzer + Bray|    4|
|         Crown Forum|    1|
|     Broadside Books|    1|
|     William Collins|   13|
|        Killer Reads|    2|
| Glencoe/Mcgraw-Hill|    1|
|       Fourth Estate|    4|
|         Definitions|    1|
|Leo Strauss, J. A...|    1|
|Harpercollins Pub...|    1|
|   Penguin Books Ltd|    1|
|      Academic Press|    6|
+--------------------+-----+
only showing top 20 rows



In [9]:
join_DF1.orderBy("start_time").show()

+----------+-----+---------+--------------------+--------------------+--------------------+-------------------+-------------------+
|      ASIN|GROUP|   FORMAT|               TITLE|              AUTHOR|           PUBLISHER|         start_time|           end_time|
+----------+-----+---------+--------------------+--------------------+--------------------+-------------------+-------------------+
|0008156123| book|hardcover|Walking Through S...|      Graham Hoyland|     William Collins|2017-01-01_16:00:00|2018-06-30_05:00:00|
|022624573X| book|hardcover|Pure Intelligence...|  Melvyn C. Usselman|University Of Chi...|2017-01-01_16:00:00|2018-06-29_23:00:00|
|026201923X| book|hardcover|Your Everyday Art...|         Lane Relyea|       The MIT Press|2017-01-01_16:00:00|2018-06-29_23:00:00|
|022618871X| book|hardcover|Plankton: Wonders...|    Christian Sardet|University Of Chi...|2017-01-01_16:00:00|2018-06-30_10:00:00|
|022626355X| book|hardcover|Gustave Caillebot...|Mary Morton, Geor...|Univer

In [18]:
# 对start_time降序
join_DF1.sort("start_time",ascending=False).show()

+----------+------+--------------+--------------------+--------------------+--------------------+-------------------+-------------------+
|      ASIN| GROUP|        FORMAT|               TITLE|              AUTHOR|           PUBLISHER|         start_time|           end_time|
+----------+------+--------------+--------------------+--------------------+--------------------+-------------------+-------------------+
|030023872X|  book|     hardcover|The Mind Is Flat:...|         Nick Chater|Yale University P...|2018-05-31_14:00:00|2018-06-30_10:00:00|
|031631613X|  book|     paperback|Less (Winner of t...|   Andrew Sean Greer|      Back Bay Books|2018-05-31_05:00:00|2018-06-30_07:00:00|
|022656147X|  book|     paperback|The Forgotten Sen...|      Pablo Maurette|University of Chi...|2018-05-29_23:00:00|2018-06-30_05:00:00|
|022656164X|  book|     hardcover|The Emotions of P...|     James M. Jasper|University of Chi...|2018-05-29_23:00:00|2018-06-30_05:00:00|
|0008102171|  book|     hardcover|